# **Automatic Summarization**

Algortimo semplice:
1. individua argomento del testo come una lista di vettori di Nasari (termine1, score1, termine2, score2, ...)
2. crea contesto raccogliendo vettori dei termini trovati al passo prima. Nel caso in cui il titolo sia troppo corto (e quindi poco informativo) posso cercare vettori di elementi che fanno parte delle definizioni dei termini trovati al passo 1
3. peso i paragrafi in base alla somma dei pesi dei termini che fanno parte del contesto con la Weighted Overlap. Mantengo solo i paragrafi che hanno un peso maggiore di una soglia

Valutazione:
- BLEU
- ROUGE

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Funzioni di utility per il preprocessing

In [61]:
# Useful to remove punctuation from first or last char of a token 
# - Esempio: senza questa funzione "It's" diventa "It" e "'s"
# - It viene eliminato perchè è una stopword, mentre 's non viene eliminato perchè non rientra nè tra le stopwords nè tra la punteggiatura
# - Con questa funzione rimuovo ' da 's e poi rimuovo nuovamente eventuali stopwords.
def remove_first_last(tokens, punct, stop):
    for i in range(len(tokens)):
        for p in punct:
            if tokens[i].startswith(p):
                tokens[i] = tokens[i][1:]
            if tokens[i].endswith(p):
                tokens[i] = tokens[i][:-1]
    tokens = [t for t in tokens if t not in stop]
    return tokens

# Remove stopwords and punctuation from the text, tokenize it and lemmatize it
def preprocess(text):
    text = text.lower()
    stop = []
    with open('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/stop_words_FULL.txt', 'r') as f:
        stop = f.read().splitlines()
    stop = set(stop)
    punct = ['.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '{', '}', '"', "'", '``', "''", '...', '’', '“', '”', '‘']
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop and t not in punct]
    lemmatizer = WordNetLemmatizer()
    tokens = list(set([lemmatizer.lemmatize(t) for t in tokens]))
    tokens = remove_first_last(tokens, punct, stop)
    return tokens

# try preprocessing
preprocess("This is a test. It's a test of the pre-processing system.")

['system', 'test', 'pre-processing']

## Parsing del file di input Nasari e creazione dizionario di vettori Nasari

In [57]:
# Parsing the Nasari file and creating a dictionary with:
# - key: word
# - value: dictionary with:
#          - key: lemma
#          - value: score
nasari = {}
with open('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/dd-small-nasari-15.txt', 'r') as f:
    lines = [line.rstrip('\n') for line in f]
    for line in lines:
        line = line.split(';')
        tmp = {}
        for lemma in line[2:]:
            lemma = lemma.split('_')
            if len(lemma) > 1:
                tmp[lemma[0]] = lemma[1]
        nasari[line[1].lower()] = tmp

# nasari

## Salvo i documenti di input

Rendo ogni documento una lista di paragrafi

In [71]:
# Save document
def save_doc(filename):
    doc = []
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            if '#' not in line and line != '': # remove empty lines and the first line with the link
                doc.append(line)
    return doc

# try save_doc
# save_doc('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/docs/Ebola-virus-disease.txt')

## Individuo topic del testo dal titolo

In [77]:
# Get title from document, considering the first line
def get_title(filename):
    doc = save_doc(filename)
    return doc[0]

# try get_title
# get_title('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/docs/Ebola-virus-disease.txt')

# Get topic words from the text checking if they are in the Nasari dictionary
def get_topic_words(text):
    tokens = preprocess(text)
    topic_words = [t for t in tokens if t in nasari.keys()]
    return topic_words

# try get_topic_words
# get_topic_words("Andy Warhol: Why the great Pop artist thought ‘Trump is sort of cheap’")

get_topic_words(get_title('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/docs/Ebola-virus-disease.txt'))

['disease', 'virus']

## Crea il contesto raccogliendo vettori dei termini trovati da *get_topic_words()* dal titolo